## **1\. What is the customer distribution across different countries ?**

In [4]:
SELECT Country, count(DISTINCT CustomerNo) as Customercount
FROM [sale_transaction-data]
group by Country
order by  Customercount desc

(38 rows affected)

Total execution time: 00:00:00.609

Country,Customercount
United Kingdom,4305
Germany,91
France,87
Spain,23
Belgium,22
Switzerland,21
Portugal,18
EIRE,13
Italy,12
Finland,12


## **2\. Customer transactions by country**

In [54]:
-- More detailed analysis including sales metrics
SELECT TOP 10 
    Country,
    COUNT(DISTINCT CustomerNo) AS customer_count,
    FORMAT(SUM(Quantity * Price), 'N2') AS total_sales,
    FORMAT(SUM(Quantity * Price) / NULLIF(COUNT(DISTINCT CustomerNo), 0), 'N2') AS avg_sales_per_customer
FROM [sale_transaction-data]
GROUP BY Country
ORDER BY SUM(Quantity * Price) DESC;

-- FORMAT(SUM(Quantity * Price), 'N2') biến số thành chuỗi, làm ORDER BY không hoạt động đúng.
-- Thay vì ORDER BY total_sales DESC, ta sắp xếp theo SUM(Quantity * Price) DESC để giữ nguyên kiểu dữ liệu số.


(10 rows affected)

Total execution time: 00:00:00.797

Country,customer_count,total_sales,avg_sales_per_customer
United Kingdom,4305,"52,524,576.47","12,200.83"
Netherlands,9,"2,151,553.59","239,061.51"
EIRE,13,"1,713,410.95","131,800.84"
Germany,91,"1,371,543.27","15,071.90"
France,87,"1,330,652.89","15,294.86"
Australia,9,"995,607.91","110,623.10"
Sweden,7,"401,879.89","57,411.41"
Switzerland,21,"361,969.25","17,236.63"
Japan,8,"293,155.44","36,644.43"
Spain,23,"281,012.27","12,217.92"


- The UK is the **largest market** in volume but has a **lower average spend per customer**.
- The Netherlands and Australia have **high-value customers** despite a **low customer count**.
- France and Germany have a **strong balance** between customers and sales.
- Smaller markets (Sweden, Switzerland, Japan, and Spain) still contribute significantly in terms of **average transaction value**.

## **3. What is the distribution of customer types (one-time, moderate, frequent) in the top 10 countries by total customers?**

- If transaction\_count per customer = 1 =\> one time customers
- If transaction\_count between 2 and 5 =\> moderate\_customers
- If transaction\_count \> 5 =\> frequent\_customers

In [56]:
WITH CustomerTransactions AS (
    SELECT 
        CustomerNo,
        Country,
        COUNT(DISTINCT TransactionNo) AS transaction_count
    FROM [sale_transaction-data]
    GROUP BY CustomerNo, Country
)
SELECT TOP 10 
    Country,
    COUNT(CustomerNo) AS total_customers,
    SUM(CASE WHEN transaction_count = 1 THEN 1 ELSE 0 END) AS one_time_customers,
    SUM(CASE WHEN transaction_count BETWEEN 2 AND 5 THEN 1 ELSE 0 END) AS moderate_customers,
    SUM(CASE WHEN transaction_count > 5 THEN 1 ELSE 0 END) AS frequent_customers,
    -- Percentage calculations
    FORMAT(100.0 * SUM(CASE WHEN transaction_count = 1 THEN 1 ELSE 0 END) / COUNT(CustomerNo), 'N2') + '%' AS pct_one_time_customers,
    FORMAT(100.0 * SUM(CASE WHEN transaction_count BETWEEN 2 AND 5 THEN 1 ELSE 0 END) / COUNT(CustomerNo), 'N2') + '%' AS pct_moderate_customers,
    FORMAT(100.0 * SUM(CASE WHEN transaction_count > 5 THEN 1 ELSE 0 END) / COUNT(CustomerNo), 'N2') + '%' AS pct_frequent_customers
FROM CustomerTransactions
GROUP BY Country
ORDER BY total_customers DESC;


(10 rows affected)

Total execution time: 00:00:00.851

Country,total_customers,one_time_customers,moderate_customers,frequent_customers,pct_one_time_customers,pct_moderate_customers,pct_frequent_customers
United Kingdom,4305,1440,2012,853,33.45%,46.74%,19.81%
Germany,91,24,44,23,26.37%,48.35%,25.27%
France,87,22,40,25,25.29%,45.98%,28.74%
Spain,23,7,14,2,30.43%,60.87%,8.70%
Belgium,22,6,11,5,27.27%,50.00%,22.73%
Switzerland,21,8,10,3,38.10%,47.62%,14.29%
Portugal,18,7,9,2,38.89%,50.00%,11.11%
EIRE,13,5,6,2,38.46%,46.15%,15.38%
Italy,12,8,2,2,66.67%,16.67%,16.67%
Finland,12,5,4,3,41.67%,33.33%,25.00%


## **4\. How can I retrieve the top or bottom N customers based on total orders or total spending, with the ability to sort by any metric and handle NULL inputs dynamically?**

The name of the procedure is: GetCustomerRanking

This procedure is designed to rank customers based on their purchasing behavior, allowing dynamic selection of:

- Top N customers
- Bottom N customers
- Sorting by any key metric (total orders, total spent, etc.)
- Handling NULL values (show all customers if no filters are applied)
- Dynamically filters by year & month (if NULL, fetches all data)

In [28]:
CREATE PROCEDURE GetCustomerRanking
    @TopN INT = NULL,          -- Number of top customers (NULL = all)
    @BottomN INT = NULL,       -- Number of bottom customers (NULL = all)
    @OrderBy NVARCHAR(50) = NULL, -- Column to sort by (NULL = no sorting)
    @Year INT = NULL,          -- Year filter (NULL = all years)
    @Month INT = NULL          -- Month filter (NULL = all months)
AS
BEGIN
    SET NOCOUNT ON;

    -- Common Table Expression (CTE) to calculate customer metrics
    WITH CustomerOrders AS (
        SELECT 
            CustomerNo,
            TransactionNo,
            SUM(Quantity * Price) AS order_value,
            YEAR(Date) AS order_year,
            MONTH(Date) AS order_month
        FROM [sale_transaction-data]
        WHERE 
            (@Year IS NULL OR YEAR(Date) = @Year)  -- Filter by year
            AND (@Month IS NULL OR MONTH(Date) = @Month)  -- Filter by month
        GROUP BY CustomerNo, TransactionNo, YEAR(Date), MONTH(Date)
    ),
    RankedCustomers AS (
        SELECT 
            CustomerNo,
            COUNT(TransactionNo) AS total_orders,
            ROUND(AVG(order_value), 2) AS avg_order_value,
            ROUND(MIN(order_value), 2) AS min_order_value,
            ROUND(MAX(order_value), 2) AS max_order_value,
            ROUND(SUM(order_value), 2) AS total_spent,
            ROW_NUMBER() OVER (
                ORDER BY 
                    CASE 
                        WHEN @OrderBy = 'total_orders' THEN COUNT(TransactionNo) 
                        WHEN @OrderBy = 'total_spent' THEN SUM(order_value) 
                        WHEN @OrderBy = 'avg_order_value' THEN AVG(order_value)
                        WHEN @OrderBy = 'min_order_value' THEN MIN(order_value)
                        WHEN @OrderBy = 'max_order_value' THEN MAX(order_value)
                        ELSE NULL 
                    END DESC
            ) AS rank_top,
            ROW_NUMBER() OVER (
                ORDER BY 
                    CASE 
                        WHEN @OrderBy = 'total_orders' THEN COUNT(TransactionNo) 
                        WHEN @OrderBy = 'total_spent' THEN SUM(order_value) 
                        WHEN @OrderBy = 'avg_order_value' THEN AVG(order_value)
                        WHEN @OrderBy = 'min_order_value' THEN MIN(order_value)
                        WHEN @OrderBy = 'max_order_value' THEN MAX(order_value)
                        ELSE NULL 
                    END ASC
            ) AS rank_bottom
        FROM CustomerOrders
        GROUP BY CustomerNo
    )
    SELECT 
        CustomerNo,
        total_orders,
        avg_order_value,
        min_order_value,
        max_order_value,
        total_spent
    FROM RankedCustomers
    WHERE 
        -- Get Top N Customers
        (@TopN IS NOT NULL AND rank_top <= @TopN)
        OR
        -- Get Bottom N Customers
        (@BottomN IS NOT NULL AND rank_bottom <= @BottomN)
        OR
        -- Get All Customers If No Limits Are Set
        (@TopN IS NULL AND @BottomN IS NULL)
    ORDER BY 
        CASE 
            WHEN @OrderBy = 'total_orders' THEN total_orders 
            WHEN @OrderBy = 'total_spent' THEN total_spent 
            WHEN @OrderBy = 'avg_order_value' THEN avg_order_value
            WHEN @OrderBy = 'min_order_value' THEN min_order_value
            WHEN @OrderBy = 'max_order_value' THEN max_order_value
            ELSE NULL 
        END DESC;
END;


Commands completed successfully.

Total execution time: 00:00:00.013

In [31]:
EXEC GetCustomerRanking @TopN = 10, @BottomN = NULL, @OrderBy = 'total_orders', @Year = 2019, @Month = NULL;


Commands completed successfully.

Total execution time: 00:00:00.744

CustomerNo,total_orders,avg_order_value,min_order_value,max_order_value,total_spent
149110,185,4719.670000,11.74,48595.51,873138.52
127480,171,1393.770000,6.19,17795.00,238334.12
178410,119,2035.650000,11.74,8893.77,242242.49
130890,90,3587.080000,74.84,15740.74,322837.35
146060,82,813.070000,24.26,2086.48,66672.06
153110,82,4601.890000,148.76,27935.24,377354.81
129710,81,1198.210000,111.42,6526.50,97055.11
146460,71,28698.750000,12.38,159953.82,2037610.99
134080,57,2715.610000,65.40,7329.82,154789.77
160290,56,6145.250000,338.16,50809.44,344134.22


## **5\. How can we identify the top N highest lifetime value customers and analyze their purchasing behavior, including average transaction value, average monthly spending, and customer lifespan?**

In [32]:
WITH CustomerMetrics AS (
    SELECT 
        CustomerNo,
        COUNT(DISTINCT TransactionNo) AS total_transactions,
        COUNT(DISTINCT ProductNo) AS unique_products_bought,
        SUM(Quantity*Price) AS lifetime_value,
        SUM(Quantity) AS total_items,
        MIN(Date) AS first_purchase,
        MAX(Date) AS last_purchase,
        COUNT(DISTINCT FORMAT(Date, 'yyyy-MM')) AS active_months  
    FROM [sale_transaction-data]
    GROUP BY CustomerNo
)
SELECT 
    cm.*,
    ROUND(lifetime_value / NULLIF(total_transactions, 0), 2) AS avg_transaction_value,
    ROUND(lifetime_value / NULLIF(active_months, 0), 2) AS avg_monthly_spend,
    ROUND(CAST(total_items AS FLOAT) / NULLIF(total_transactions, 0), 2) AS avg_items_per_transaction,
    DATEDIFF(DAY, first_purchase, last_purchase) AS customer_age_days  -- days from first purchase to last purchase
FROM CustomerMetrics cm
ORDER BY lifetime_value DESC
OFFSET 0 ROWS FETCH NEXT 10 ROWS ONLY;


(10 rows affected)

Total execution time: 00:00:05.653

CustomerNo,total_transactions,unique_products_bought,lifetime_value,total_items,first_purchase,last_purchase,active_months,avg_transaction_value,avg_monthly_spend,avg_items_per_transaction,customer_age_days
146460,73,700,2112282.03,197420,2018-12-20,2019-12-08,13,28935.370000,162483.230000,"2704,38",353
164460,2,3,1002741.57,80997,2019-05-18,2019-12-09,2,501370.790000,501370.790000,"40498,5",205
149110,198,1780,914633.52,80383,2018-12-01,2019-12-08,13,4619.360000,70356.420000,"405,97",372
124150,20,442,900545.54,77669,2019-01-06,2019-11-15,10,45027.280000,90054.550000,"3883,45",313
181020,60,150,897137.36,64124,2018-12-07,2019-12-09,12,14952.290000,74761.450000,"1068,73",367
174500,46,124,891438.53,69993,2018-12-07,2019-12-01,12,19379.100000,74286.540000,"1521,59",359
123460,1,1,840113.80,74215,2019-01-18,2019-01-18,1,840113.800000,840113.800000,74215,0
141560,54,711,694965.02,57845,2018-12-03,2019-11-30,12,12869.720000,57913.750000,"1071,2",362
136940,51,532,646116.78,63860,2018-12-01,2019-12-06,13,12668.960000,49701.290000,"1252,16",370
175110,31,453,639006.19,64549,2018-12-01,2019-12-07,13,20613.100000,49154.320000,"2082,23",371


## **6\. How many times has the price changed for each product?**

In [32]:
WITH PriceHistory AS (
    SELECT  
        Productname, 
        Price, 
        Date,
        LAG(Price) OVER (PARTITION BY Productname ORDER BY Date) AS PrevPrice
    FROM [sale_transaction-data]
),
PriceChangeFlag AS (
    SELECT  
        *,
        CASE 
            WHEN Price = PrevPrice THEN 0 
            ELSE 1 
        END AS PriceChange
    FROM PriceHistory
)
SELECT TOP 10
    Productname,
    SUM(PriceChange) AS TotalPriceChanges
FROM PriceChangeFlag
GROUP BY Productname
ORDER BY TotalPriceChanges DESC;  -- Sắp xếp theo số lần thay đổi giá giảm dần


(10 rows affected)

Total execution time: 00:00:04.546

Productname,TotalPriceChanges
Cream Hanging Heart T-Light Holder,961
Jumbo Bag Red Retrospot,892
Popcorn Holder,664
Regency Cakestand 3 Tier,645
Jam Making Set With Jars,580
Party Bunting,579
Pack Of 72 Retrospot Cake Cases,560
Lunch Bag Red Retrospot,546
Red Retrospot Charlotte Bag,535
Recipe Box Pantry Yellow Design,523


## **7\. Analyze customers who have transactions for at least `n` consecutive days, ensuring that the total number of transactions within these `n` days is greater than `m`**

In [48]:
DECLARE @n INT = 4, @m INT = 3;

WITH cte1 AS (
    SELECT 
        CustomerNo,
        Date,
        -- Rank transactions per customer based on date
        DENSE_RANK() OVER (PARTITION BY CustomerNo ORDER BY Date) AS transaction_rank,
        -- Identify the start of a transaction streak by subtracting the rank from the date
        DATEADD(DAY, -DENSE_RANK() OVER (PARTITION BY CustomerNo ORDER BY Date), Date) AS transaction_streak_start
    FROM [sale_transaction-data]
)
SELECT 
    CustomerNo,
    MIN(Date) AS start_date,  -- First transaction date in the streak
    MAX(Date) AS end_date,  -- Last transaction date in the streak
    COUNT(DISTINCT Date) AS unique_days,  -- Unique transaction days in the streak
    COUNT(*) AS total_transactions  -- Total number of transactions in the streak
FROM cte1
GROUP BY CustomerNo, transaction_streak_start
HAVING 
    COUNT(*) >=@n -- Ensure the streak has at least @min_transactions
    AND COUNT(DISTINCT Date) >= @m -- Ensure at least @min_unique_days
ORDER BY CustomerNo;


(29 rows affected)

Total execution time: 00:00:00.976

CustomerNo,start_date,end_date,unique_days,total_transactions
127480,2019-11-06,2019-11-08,3,266
127480,2018-12-05,2018-12-10,6,535
127480,2019-10-05,2019-10-07,3,43
127480,2019-08-03,2019-08-05,3,30
127480,2019-06-21,2019-06-23,3,37
127480,2019-11-22,2019-11-25,4,696
127480,2019-06-06,2019-06-08,3,66
127480,2019-11-15,2019-11-18,4,378
127480,2019-05-10,2019-05-12,3,69
129210,2019-09-14,2019-09-16,3,65


## **8\. How many customers fall into each spending segment (\<P25, P25 - P50, P50 - P75, \>P75)?**

<span style="color: #008000;">--Instead of setting arbitrary sales ranges, I use PERCENTILE calculations to dynamically group customers based on actual sales distribution.</span>

In [17]:
-- Declare variables to store percentile values
DECLARE @p25 DECIMAL(18,2);
DECLARE @p50 DECIMAL(18,2);
DECLARE @p75 DECIMAL(18,2);

-- Calculate percentiles dynamically
WITH sales_distribution AS (
    SELECT 
        CustomerNo, 
        SUM(Quantity * Price) AS total_sales
    FROM [sale_transaction-data]
    GROUP BY CustomerNo
),
ranked_sales AS (
    SELECT 
        total_sales, 
        PERCENT_RANK() OVER (ORDER BY total_sales ASC) AS percentile_rank
    FROM sales_distribution
),
percentiles AS (
    SELECT 
        MAX(CASE WHEN percentile_rank <= 0.25 THEN total_sales END) AS p25_sales,
        MAX(CASE WHEN percentile_rank <= 0.50 THEN total_sales END) AS p50_sales,
        MAX(CASE WHEN percentile_rank <= 0.75 THEN total_sales END) AS p75_sales
    FROM ranked_sales
)
-- Assign percentile values to variables
SELECT 
    @p25 = p25_sales,
    @p50 = p50_sales,
    @p75 = p75_sales
FROM percentiles;

-- Create customer segments based on purchase value percentiles
WITH cte1 AS (
    SELECT '<P25' AS segment_sales, 
           FORMAT(0, '#,##0.00') + ' - ' + FORMAT(@p25, '#,##0.00') AS segment_sales_range,
           0 AS a, @p25 AS b
    UNION ALL
    SELECT 'P25 - P50', 
           FORMAT(@p25, '#,##0.00') + ' - ' + FORMAT(@p50, '#,##0.00'),
           @p25, @p50
    UNION ALL
    SELECT 'P50 - P75', 
           FORMAT(@p50, '#,##0.00') + ' - ' + FORMAT(@p75, '#,##0.00'),
           @p50, @p75
    UNION ALL
    SELECT '>P75', 
           FORMAT(@p75, '#,##0.00') + ' - ' + 'Max',
           @p75, 999999999
),
cte2 AS (
    SELECT 
        CustomerNo, 
        COUNT(DISTINCT ProductNo) AS unique_products_bought, 
        SUM(Quantity * Price) AS total_value,
        COUNT(DISTINCT TransactionNo) AS total_orders
    FROM [sale_transaction-data]
    GROUP BY CustomerNo
)
-- Assign customers to segments based on total purchase value
SELECT 
    t1.segment_sales, 
    t1.segment_sales_range, -- Display segment range
    COUNT(t2.CustomerNo) AS total_customers,
    ISNULL(SUM(t2.total_value), 0) AS total_order_value,
    ISNULL(SUM(t2.total_orders), 0) AS total_orders
FROM cte1 t1
LEFT JOIN cte2 t2 ON t2.total_value BETWEEN t1.a AND t1.b
GROUP BY t1.segment_sales, t1.segment_sales_range
ORDER BY 
    CASE 
        WHEN t1.segment_sales = '<P25' THEN 1
        WHEN t1.segment_sales = 'P25 - P50' THEN 2
        WHEN t1.segment_sales = 'P50 - P75' THEN 3
        WHEN t1.segment_sales = '>P75' THEN 4
    END;



Warning: Null value is eliminated by an aggregate or other SET operation.

(4 rows affected)

Total execution time: 00:00:01.043

segment_sales,segment_sales_range,total_customers,total_order_value,total_orders
<P25,"0.00 - 1,838.40",1180,1124309.65,1607
P25 - P50,"1,838.40 - 4,826.72",1180,3694760.13,2469
P50 - P75,"4,826.72 - 11,898.92",1180,9154670.20,4211
>P75,"11,898.92 - Max",1181,49010716.40,11510


## **9\. Analyze the top 20 product pairs that are most frequently purchased together in the same transaction**

In [2]:
--Analyze the top 20 product pairs that are most frequently purchased together in the same transaction
WITH cte AS (
    SELECT  
        TransactionNo, 
        ProductName
    FROM [sale_transaction-data]
), 
cte1 AS (
    SELECT 
        a1.TransactionNo, 
        a1.ProductName AS Product_1, 
        a2.ProductName AS Product_2
    FROM cte a1 
    INNER JOIN cte a2 
        ON a1.TransactionNo = a2.TransactionNo 
        AND a1.ProductName < a2.ProductName 
)
SELECT TOP 20 
    Product_1, 
    Product_2, 
    COUNT(*) AS count_times 
FROM cte1
GROUP BY Product_1, Product_2 
ORDER BY COUNT(*) DESC;


(20 rows affected)

Total execution time: 00:01:49.600

Product_1,Product_2,count_times
Green Regency Teacup And Saucer,Pink Regency Teacup And Saucer,894
Jumbo Bag Pink Polkadot,Jumbo Bag Red Retrospot,863
Green Regency Teacup And Saucer,Roses Regency Teacup And Saucer,845
Jumbo Bag Red Retrospot,Jumbo Storage Bag Suki,747
Lunch Bag Black Skull,Lunch Bag Red Retrospot,699
Lunch Bag Red Retrospot,Lunch Bag Suki Design,695
Jumbo Bag Red Retrospot,Jumbo Shopper Vintage Red Paisley,694
Alarm Clock Bakelike Green,Alarm Clock Bakelike Red,690
Pink Regency Teacup And Saucer,Roses Regency Teacup And Saucer,678
Lunch Bag Black Skull,Lunch Bag Suki Design,647


## **10\. Analyze the time gap between a customer's first purchase and their second purchase**

categorizing it into three ranges: \<7 days, 7-30 days, and \>30 days

In [48]:
--Analyze the time gap between a customer's first purchase and their second purchase, categorizing it into three ranges: <7 days, 7-30 days, and >30 days.
WITH cte01 AS (
    SELECT
        CustomerNo,
        MIN(Date) AS order_date_1st  -- First purchase date
    FROM [sale_transaction-data]
    GROUP BY CustomerNo
),
cte02 AS (
    SELECT
        a.CustomerNo,
        a.order_date_1st,
        MIN(b.Date) AS order_date_2nd  -- Find the second purchase date
    FROM cte01 a
    JOIN [sale_transaction-data] b 
        ON a.CustomerNo = b.CustomerNo 
        AND b.Date > a.order_date_1st
    GROUP BY a.CustomerNo, a.order_date_1st
)
SELECT
    YEAR(order_date_1st) AS purchase_1st_year,
    MONTH(order_date_1st) AS purchase_1st_month,
    CASE
        WHEN DATEDIFF(DAY, order_date_1st, order_date_2nd) < 7 THEN '< 07 days'
        WHEN DATEDIFF(DAY, order_date_1st, order_date_2nd) BETWEEN 7 AND 30 THEN '07 - 30 days'
        ELSE '> 30 days'
    END AS purchase_gap,
    COUNT(DISTINCT CustomerNo) AS num_custs
FROM cte02
WHERE order_date_2nd IS NOT NULL 
GROUP BY
    YEAR(order_date_1st), MONTH(order_date_1st),
    CASE
        WHEN DATEDIFF(DAY, order_date_1st, order_date_2nd) < 7 THEN '< 07 days'
        WHEN DATEDIFF(DAY, order_date_1st, order_date_2nd) BETWEEN 7 AND 30 THEN '07 - 30 days'
        ELSE '> 30 days'
    END
ORDER BY purchase_1st_year, purchase_1st_month, purchase_gap;


(36 rows affected)

Total execution time: 00:00:00.789

purchase_1st_year,purchase_1st_month,purchase_gap,num_custs
2018,12,< 07 days,88
2018,12,> 30 days,600
2018,12,07 - 30 days,199
2019,1,< 07 days,12
2019,1,> 30 days,308
2019,1,07 - 30 days,73
2019,2,< 07 days,8
2019,2,> 30 days,275
2019,2,07 - 30 days,54
2019,3,< 07 days,9


## **11\. For each week, how many customers are classified as retained (purchased within 4 weeks), returning (purchased after more than 4 weeks), or new (first-time buyers)**

_Situation 1 : Customers from different years are always new_

In [45]:
-- Step 1: Extract distinct customer IDs along with their purchase week and year  
WITH cte_01 AS (
    SELECT DISTINCT 
        CustomerNo,
        DATEPART(WEEK, Date) AS [week],  
        YEAR(Date) AS [year]  
    FROM [sale_transaction-data]
),

-- Step 2: Calculate the previous purchase week and year for each customer 
-- and determine the difference in weeks between consecutive purchases  
cte_02 AS (
    SELECT 
        CustomerNo,
        [week],
        [year],
        LAG([week]) OVER (PARTITION BY CustomerNo ORDER BY [year], [week]) AS lag_week,  -- Previous purchase week
        LAG([year]) OVER (PARTITION BY CustomerNo ORDER BY [year], [week]) AS lag_year,  -- Previous purchase year
        [week] - LAG([week]) OVER (PARTITION BY CustomerNo ORDER BY [year], [week]) AS week_diff  -- Difference in weeks
    FROM cte_01
),

-- Step 3: Categorize customers into 'new', 'retained', or 'returning'  
cte_03 AS (
    SELECT 
        CustomerNo,
        [week],
        [year],
        CASE
            WHEN lag_year IS NULL OR lag_year <> [year] THEN 'new'  -- First-time customers or those in a new year
            WHEN week_diff <= 4 THEN 'retained'  -- Customers who made another purchase within 4 weeks
            WHEN week_diff > 4 THEN 'returning'  -- Customers who purchased again after more than 4 weeks
        END AS cust_type
    FROM cte_02
)

-- Step 4: Count the number of customers in each category per week  
SELECT 
    [week],
    [year],
    cust_type,
    COUNT(CustomerNo) AS num_customers  -- Count customers in each category
FROM cte_03
GROUP BY [week], [year], cust_type
ORDER BY [year], [week], cust_type;  -- Sort by year, week, and customer type


(153 rows affected)

Total execution time: 00:00:00.732

week,year,cust_type,num_customers
48,2018,new,101
49,2018,new,450
49,2018,retained,27
50,2018,new,279
50,2018,retained,104
51,2018,new,179
51,2018,retained,107
52,2018,new,17
52,2018,retained,8
1,2019,new,87


_Situation 2: Customers from different years can still be retained/returning_

In [46]:
-- Step 1: Extract unique customer transactions with their respective week and year
WITH cte_01 AS (
    SELECT DISTINCT 
        CustomerNo,
        DATEPART(WEEK, Date) AS [week],  -- Extract week number from transaction date
        YEAR(Date) AS [year]  -- Extract year from transaction date
    FROM [sale_transaction-data]
),

-- Step 2: Calculate the previous week's and year's transactions for each customer
cte_02 AS (
    SELECT 
        CustomerNo,
        [week],
        [year],
        -- Get the last week's transaction for the same customer
        LAG([week]) OVER (PARTITION BY CustomerNo ORDER BY [year], [week]) AS lag_week,
        -- Get the last year's transaction for the same customer
        LAG([year]) OVER (PARTITION BY CustomerNo ORDER BY [year], [week]) AS lag_year,
        -- Calculate the difference in weeks between the current and previous transaction
        [week] - LAG([week]) OVER (PARTITION BY CustomerNo ORDER BY [year], [week]) AS week_diff
    FROM cte_01
),

-- Step 3: Classify customers into "new," "retained," or "returning" based on purchase patterns
cte_03 AS (
    SELECT 
        CustomerNo,
        [week],
        [year],
        CASE
            -- If the customer has a previous purchase in a different year, check if they are retained or returning
            WHEN lag_year IS NOT NULL AND lag_year < [year] AND week_diff <= 4 THEN 'retained'
            WHEN lag_year IS NOT NULL AND lag_year < [year] AND week_diff > 4 THEN 'returning'
            -- If there is no previous transaction, classify them as "new"
            ELSE 'new'
        END AS cust_type
    FROM cte_02
)

-- Step 4: Count the number of customers in each category per week and year
SELECT 
    [week],  
    [year],  
    cust_type,  -- Customer type (new, retained, returning)
    COUNT(CustomerNo) AS num_customers  -- Count of customers in each category
FROM cte_03
GROUP BY [week], [year], cust_type
ORDER BY [year], [week], cust_type;


(104 rows affected)

Total execution time: 00:00:00.611

week,year,cust_type,num_customers
48,2018,new,101
49,2018,new,477
50,2018,new,383
51,2018,new,286
52,2018,new,25
1,2019,new,33
1,2019,retained,54
2,2019,new,158
2,2019,retained,105
3,2019,new,144


## **12\. How many customers made a purchase in month N and returned in the following months?**

In [7]:
DECLARE @year INT = 2019;  -- Define the year for analysis
DECLARE @n INT = 1;        -- Define the starting month 

-- Step 1: Identify customers who made a purchase in the given month (N)
WITH FirstPurchaseCustomers AS (
    SELECT DISTINCT CustomerNo
    FROM [sale_transaction-data]
    WHERE MONTH(Date) = @n AND YEAR(Date) = @year
)

-- Step 2: Count how many of these customers returned in subsequent months
SELECT 
    MONTH(Date) AS [month],                      
    COUNT(DISTINCT CustomerNo) AS num_cust       -- Count unique returning customers
FROM [sale_transaction-data]
WHERE 
    CustomerNo IN (SELECT CustomerNo FROM FirstPurchaseCustomers) 
    AND MONTH(Date) > @n  
    AND YEAR(Date) = @year  
GROUP BY MONTH(Date)  
ORDER BY [month];  


(11 rows affected)

Total execution time: 00:00:00.806

month,num_cust
2,277
3,343
4,309
5,361
6,336
7,310
8,305
9,341
10,337
11,409
